In [2]:
import pandas as pd
import json
import csv
import re

In [3]:
# Paths to input and output files
input_files = {
    'credits': 'credits.csv',
    'keywords': 'keywords.csv',
    'movies_metadata': 'movies_metadata.csv'
}

In [4]:
output_files = {
    'credits': 'transformed_credits.csv',
    'keywords': 'transformed_keywords.csv',
    'movies_metadata': 'transformed_movies_metadata.csv',
    'genres': 'transformed_genres.csv',
    'production_companies': 'transformed_production_companies.csv',
    'production_countries': 'transformed_production_countries.csv'
}


In [15]:

# Function to convert JSON-like strings to JSON and handle escaping
def json_to_str(json_str):
    if isinstance(json_str, str):
        try:
            json_obj = json.loads(json_str.replace("'", '"'))
            return json.dumps(json_obj)
        except json.JSONDecodeError:
            return json_str
    return json_str

# Function to expand JSON-like strings to separate rows
def expand_json(json_str):
    if isinstance(json_str, str):
        try:
            json_obj = json.loads(json_str.replace("'", '"'))
            return json_obj
        except json.JSONDecodeError:
            return []
    return []

# Function to remove non-ASCII characters from a string
def remove_non_ascii(text):
    if isinstance(text, str):
        return re.sub(r'[^\x00-\x7F]+', '', text)
    return text

# Function to remove trailing single quotes from a string
def remove_trailing_quote(text):
    if isinstance(text, str):
        return re.sub(r"'\s*$", '', text)
    return text

# Function to remove double quotes within a string
def remove_double_quotes(text):
    if isinstance(text, str):
        return text.replace('"', '')
    return text

# Function to remove single quotes within a string
def remove_single_quotes(text):
    if isinstance(text, str):
        return text.replace("'", "")
    return text

# Function to clean boolean fields
def clean_boolean_field(text):
    if isinstance(text, str):
        text = text.lower()
        if text in ['true', 't', 'yes', 'y', '1']:
            return True
        elif text in ['false', 'f', 'no', 'n', '0']:
            return False
    return None

# Load and transform movies_metadata.csv
movies_metadata = pd.read_csv(input_files['movies_metadata'], low_memory=False, encoding='utf-8')

# Rename 'id' column to 'movie_id'
movies_metadata.rename(columns={'id': 'movie_id'}, inplace=True)

# Filter out rows with non-numeric movie_id
movies_metadata = movies_metadata[movies_metadata['movie_id'].apply(lambda x: str(x).isdigit())]

# Convert movie_id to integers
movies_metadata['movie_id'] = movies_metadata['movie_id'].astype(int)

# Convert Boolean columns and clean invalid values
movies_metadata['adult'] = movies_metadata['adult'].astype(str).apply(clean_boolean_field)

# Replace empty strings in numerical columns with default values
numerical_columns = ['budget', 'revenue', 'runtime', 'popularity', 'vote_average', 'vote_count']
for column in numerical_columns:
    movies_metadata[column] = pd.to_numeric(movies_metadata[column], errors='coerce').fillna(0)

# Handle text fields with potential newlines, special characters, and non-ASCII characters
text_columns = ['overview', 'tagline', 'original_title', 'title']
for column in text_columns:
    movies_metadata[column] = movies_metadata[column].astype(str).str.replace('\n', ' ').replace('\r', ' ')
    movies_metadata[column] = movies_metadata[column].apply(remove_non_ascii)
    movies_metadata[column] = movies_metadata[column].apply(remove_trailing_quote)
    movies_metadata[column] = movies_metadata[column].apply(remove_double_quotes)

movies_metadata.drop_duplicates(subset='movie_id', inplace=True)
movies_metadata['release_date'].fillna('1900-01-01', inplace=True)
movies_metadata['release_date'] = pd.to_datetime(movies_metadata['release_date'], errors='coerce')
movies_metadata['original_language'] = movies_metadata['original_language'].str.lower()

# Convert belongs_to_collection to JSON
movies_metadata['belongs_to_collection'] = movies_metadata['belongs_to_collection'].apply(json_to_str)

# Remove the video and spoken_languages columns
columns_to_remove = ['video', 'spoken_languages']
movies_metadata = movies_metadata.drop(columns=[col for col in columns_to_remove if col in movies_metadata.columns])

# Normalize and remove JSON-like columns
json_columns = ['genres', 'production_companies', 'production_countries']
normalized_data = {}

# Function to normalize a JSON field from movies_metadata
def normalize_json_field(df, json_field, field_names):
    expanded_rows = []
    for index, row in df.iterrows():
        json_list = expand_json(row[json_field])
        for item in json_list:
            normalized_item = {'movie_id': row['movie_id']}
            for field in field_names:
                normalized_item[field] = item.get(field)
            expanded_rows.append(normalized_item)
    return pd.DataFrame(expanded_rows)

for column in json_columns:
    if column in movies_metadata.columns:
        normalized_data[column] = normalize_json_field(movies_metadata, column, ['id', 'name'])
        movies_metadata = movies_metadata.drop(columns=[column])

# Save the transformed movies_metadata.csv with quoting all text fields and escape characters
with open(output_files['movies_metadata'], mode='w', newline='', encoding='utf-8') as file:
    movies_metadata.to_csv(file, index=False, quoting=csv.QUOTE_ALL, escapechar='\\', na_rep='NULL')

# Save the normalized data to separate CSV files
for column in json_columns:
    if column in normalized_data:
        with open(output_files[column], mode='w', newline='', encoding='utf-8') as file:
            normalized_data[column].to_csv(file, index=False, quoting=csv.QUOTE_ALL, escapechar='\\', na_rep='NULL')

# Get the list of valid movie IDs
valid_movie_ids = set(movies_metadata['movie_id'])

# Load and transform credits.csv
credits = pd.read_csv(input_files['credits'])

# Filter credits to only include rows with valid movie IDs
credits = credits[credits['id'].apply(lambda x: str(x).isdigit())]
credits['id'] = credits['id'].astype(int)
credits = credits[credits['id'].isin(valid_movie_ids)]

# Rename 'id' column to 'movie_id' in credits DataFrame
credits.rename(columns={'id': 'movie_id'}, inplace=True)

# Debugging: Print unique movie IDs
print("Unique movie IDs in credits.csv after filtering:", credits['movie_id'].unique())
print("Unique movie IDs in movies_metadata.csv:", movies_metadata['movie_id'].unique())

# Expand cast and crew columns
expanded_cast = []
expanded_crew = []

for index, row in credits.iterrows():
    cast_list = expand_json(row['cast'])
    crew_list = expand_json(row['crew'])
    movie_id = row['movie_id']
    
    for cast in cast_list:
        expanded_cast.append({
            'movie_id': movie_id,
            'cast_id': cast.get('cast_id'),
            'character': cast.get('character'),
            'credit_id': cast.get('credit_id'),
            'gender': cast.get('gender'),
            'id': cast.get('id'),
            'name': cast.get('name'),
            'order': cast.get('order'),
            'profile_path': cast.get('profile_path')
        })
    
    for crew in crew_list:
        expanded_crew.append({
            'movie_id': movie_id,
            'credit_id': crew.get('credit_id'),
            'department': crew.get('department'),
            'gender': crew.get('gender'),
            'id': crew.get('id'),
            'job': crew.get('job'),
            'name': crew.get('name'),
            'profile_path': crew.get('profile_path')
        })

# Create DataFrames for expanded cast and crew
expanded_cast_df = pd.DataFrame(expanded_cast)
expanded_crew_df = pd.DataFrame(expanded_crew)

# Remove duplicates based on movie_id and cast_id for cast
expanded_cast_df.drop_duplicates(subset=['movie_id', 'cast_id'], inplace=True)

# Remove duplicates based on movie_id and credit_id for crew
expanded_crew_df.drop_duplicates(subset=['movie_id', 'credit_id'], inplace=True)

# Normalize production_countries field
# Function to handle JSON fields and expand them
def normalize_json_field(df, field, fields):
    normalized_data = []
    for _, row in df.iterrows():
        movie_id = row['movie_id']
        try:
            json_list = eval(row[field]) if isinstance(row[field], str) else []
            for item in json_list:
                normalized_data.append({**item, 'movie_id': movie_id})
        except (TypeError, ValueError):
            continue
    normalized_df = pd.DataFrame(normalized_data, columns=fields + ['movie_id'])
    return normalized_df

# Load movies_metadata.csv
movies_metadata = pd.read_csv(input_files['movies_metadata'], low_memory=False)

# Rename 'id' column to 'movie_id'
movies_metadata.rename(columns={'id': 'movie_id'}, inplace=True)

# Normalize production_countries field
if 'production_countries' in movies_metadata.columns:
    production_countries_fields = ['iso_3166_1', 'name']
    production_countries_df = normalize_json_field(movies_metadata, 'production_countries', production_countries_fields)
    
    # Remove the 'id' column if it exists
    if 'id' in production_countries_df.columns:
        production_countries_df.drop(columns=['id'], inplace=True)
    
    # Remove duplicates
    production_countries_df.drop_duplicates(subset=['movie_id', 'iso_3166_1'], inplace=True)
    
    # Save the cleaned data to CSV
    production_countries_df.to_csv(output_files['production_countries'], index=False, quoting=csv.QUOTE_ALL, escapechar='\\')

print("Production countries have been transformed and saved.")

# Save transformed data to CSV files with quoting all text fields and escape characters
with open(output_files['credits'], mode='w', newline='', encoding='utf-8') as file:
    credits.to_csv(file, index=False, quoting=csv.QUOTE_ALL, escapechar='\\', na_rep='NULL')
with open('transformed_cast.csv', mode='w', newline='', encoding='utf-8') as file:
    expanded_cast_df.to_csv(file, index=False, quoting=csv.QUOTE_ALL, escapechar='\\', na_rep='NULL')
with open('transformed_crew.csv', mode='w', newline='', encoding='utf-8') as file:
    expanded_crew_df.to_csv(file, index=False, quoting=csv.QUOTE_ALL, escapechar='\\', na_rep='NULL')

print("All files have been transformed and saved.")


Unique movie IDs in credits.csv after filtering: [   862   8844  15602 ...  67758 227506 461257]
Unique movie IDs in movies_metadata.csv: [   862   8844  15602 ...  67758 227506 461257]
Production countries have been transformed and saved.
All files have been transformed and saved.
